In [2]:
import pandas as pd
# 웹 서버에 요청을 보내고 응답 메시지 받는 기능
import requests
# 크롤링 기능
from bs4 import BeautifulSoup as bs
# 웹브라우저를 제어
from selenium import webdriver
# 브라우저의 태그를 검색
from selenium.webdriver.common.by import By
# 키보드의 특수 이벤트(ENTER, DELETE, CLEAR, ...)
from selenium.webdriver.common.keys import Keys

- 네이버 증권
    - 종목코드를 이용하여 검색
        - url의 규칙 확인
    - 뉴스 탭에 더보기 버튼을 선택
        - 클릭 이벤트 발생하여 페이지 이동
        - url의 규칙 확인
    - 뉴스 리스트에 있는 하이퍼링크의 주소를 모두 리스트의 형태로 저장
        - requests라이브러리를 이용하여 해당 주소로 요청
            - 정보를 확인
            - 기사의 내용이 모두 응답 메시지에 존재한다면 제목, 본문 내용으로 1차원 리스트 생성
            - 모든 기사 내용을 리스트에 추가(2차원 리스트)
            - pandas 이용하여 데이터프레임을 생성
            - csv 파일로 저장 (파일의 이름에 현재 시간을 대입해서 파일 중복 방지)

In [3]:
url = 'https://finance.naver.com/item/main.naver?code='

code = '012450'

In [4]:
# 웹 브라우저 오픈
driver = webdriver.Chrome()

In [5]:
# 특정 주소를 요청을 보낸다.
driver.get(url+code)

In [6]:
# driver에서 html코드를 불러와서(웹 브라우저에 page_source)
# BeautifulSoup을 이용하여 데이터를 파싱(class 생성)
soup = bs(driver.page_source, 'html.parser')

In [7]:
len(driver.page_source)

231923

In [8]:
# div 태그 중에 class값이 news_section 인 태그의 개수를 확인
len(
    soup.find_all(
        'div',
        attrs = {
            'class' : 'news_section'
        }
    )
)

1

In [9]:
# iframe 태그중 class가 news_section이라는 태그를 추출
div_data = soup.find(
    'div',
    attrs = {
        'class' : 'news_section'
    }
)

In [10]:
# div_data에서 li태그를 모두 검색
li_list = div_data.find_all('li')

In [11]:
len(li_list)

10

In [12]:
# li_list의 각 원소들의 첫번째 a태그를 선택하여 href의 속성값을 이용하여 새로운 리스트 생성
href_list = []

for li_data in li_list:
    # li_data에서 첫번째 a태그를 추출 -> href 속성의 값을 추출
    data = li_data.find('a')['href']
    # href_list에 추가
    href_list.append(data)

In [13]:
href_list

['/item/news_read.naver?article_id=0005105604&office_id=015&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0001201704&office_id=215&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0003292846&office_id=030&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0005165274&office_id=008&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0004460873&office_id=011&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0000429490&office_id=374&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0015262985&office_id=001&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0000099084&office_id=586&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0004460830&office_id=011&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0005165198&office_id=008&code=012450&sm=entity_id.basic']

In [14]:
base_url = "https://finance.naver.com"

In [15]:
href_list[0]

'/item/news_read.naver?article_id=0005105604&office_id=015&code=012450&sm=entity_id.basic'

In [16]:
res = requests.get(base_url+href_list[0])

In [17]:
print(res.text)

<SCRIPT>top.location.href='https://n.news.naver.com/mnews/article/015/0005105604';</SCRIPT>



In [18]:
# selenium을 이용해서 base_url + href_list[0] 에 요청
driver.get(base_url + href_list[0])

In [19]:
# html문서를 bs를 이용하여 데이터 파싱
news_soup = bs(driver.page_source, 'html.parser')

In [20]:
# h2태그중 id가 'title_area'인 태그를 선택하고 텍스트 추출
news_title = news_soup.find(
    'h2',
    attrs={
        'id' : 'title_area'
    }
).get_text()
news_title

"'외인 팔자' 코스피, 상승폭 반납…2570선 보합권 등락"

In [21]:
# div 태그중에 id가 'newsct_article'인 태그를 선택하고 텍스트 추출
news_content = news_soup.find(
    'div',
    attrs={
        'id' : 'newsct_article'
    }
).get_text().strip().replace('\n', '')

news_content

'사진=연합뉴스13일 코스피지수가 외국인 투자자의 팔자에 장중 상승폭을 반납했다.이날 오후 1시18분 현재 코스피지수는 전일 대비 0.92포인트(0.04%) 오른 2575.74를 기록 중이다. 0.9% 상승 출발한 코스피지수는 이날 상승폭을 2600선까지 높이기도 했으나 외인과 금융투자에서 매물이 출회되면서 상단이 제한됐다.이 시각 현재 외국인은 유가증권시장에서 995억원 순매도 중이다. 금융투자도 1312억원 팔고 있다. 개인은 149억원 매수우위다. 기관은 3억원 매수우위다.코스피 시가총액 상위기업들은 혼조세다. SK하이닉스가 장중 하락으로 전환한 가운데 LG에너지솔루션, 삼성바이오로직스, 현대차, POSCO홀딩스가 내림세다. 반면 삼성전자, 기아, 네이버 등은 오르고 있다.한화에어로스페이스는 증권가 목표가 상향에 6%대 강세다. LS는 호반그룹이 지분을 매입했다는 소식에 18%대 뛰고 있다. 현대로템은 유럽 방위비 증강 기대감에 신고가를 다시 썼다. 경영권 분쟁에 다시 불이 붙어 전날 급등했던 고려아연은 이날 현재 9%대 급락세다.코스닥지수는 장중 하락으로 전환했다. 이 시각 현재 코스닥지수는 0.42% 떨어진 726.44를 기록 중이다.'

In [22]:
news_data = []
news_data.append(news_title)
news_data.append(news_content)

In [23]:
news_data

["'외인 팔자' 코스피, 상승폭 반납…2570선 보합권 등락",
 '사진=연합뉴스13일 코스피지수가 외국인 투자자의 팔자에 장중 상승폭을 반납했다.이날 오후 1시18분 현재 코스피지수는 전일 대비 0.92포인트(0.04%) 오른 2575.74를 기록 중이다. 0.9% 상승 출발한 코스피지수는 이날 상승폭을 2600선까지 높이기도 했으나 외인과 금융투자에서 매물이 출회되면서 상단이 제한됐다.이 시각 현재 외국인은 유가증권시장에서 995억원 순매도 중이다. 금융투자도 1312억원 팔고 있다. 개인은 149억원 매수우위다. 기관은 3억원 매수우위다.코스피 시가총액 상위기업들은 혼조세다. SK하이닉스가 장중 하락으로 전환한 가운데 LG에너지솔루션, 삼성바이오로직스, 현대차, POSCO홀딩스가 내림세다. 반면 삼성전자, 기아, 네이버 등은 오르고 있다.한화에어로스페이스는 증권가 목표가 상향에 6%대 강세다. LS는 호반그룹이 지분을 매입했다는 소식에 18%대 뛰고 있다. 현대로템은 유럽 방위비 증강 기대감에 신고가를 다시 썼다. 경영권 분쟁에 다시 불이 붙어 전날 급등했던 고려아연은 이날 현재 9%대 급락세다.코스닥지수는 장중 하락으로 전환했다. 이 시각 현재 코스닥지수는 0.42% 떨어진 726.44를 기록 중이다.']

In [24]:
news_dict = {}
news_dict['title'] = news_title
news_dict['content'] = news_content
news_dict

{'title': "'외인 팔자' 코스피, 상승폭 반납…2570선 보합권 등락",
 'content': '사진=연합뉴스13일 코스피지수가 외국인 투자자의 팔자에 장중 상승폭을 반납했다.이날 오후 1시18분 현재 코스피지수는 전일 대비 0.92포인트(0.04%) 오른 2575.74를 기록 중이다. 0.9% 상승 출발한 코스피지수는 이날 상승폭을 2600선까지 높이기도 했으나 외인과 금융투자에서 매물이 출회되면서 상단이 제한됐다.이 시각 현재 외국인은 유가증권시장에서 995억원 순매도 중이다. 금융투자도 1312억원 팔고 있다. 개인은 149억원 매수우위다. 기관은 3억원 매수우위다.코스피 시가총액 상위기업들은 혼조세다. SK하이닉스가 장중 하락으로 전환한 가운데 LG에너지솔루션, 삼성바이오로직스, 현대차, POSCO홀딩스가 내림세다. 반면 삼성전자, 기아, 네이버 등은 오르고 있다.한화에어로스페이스는 증권가 목표가 상향에 6%대 강세다. LS는 호반그룹이 지분을 매입했다는 소식에 18%대 뛰고 있다. 현대로템은 유럽 방위비 증강 기대감에 신고가를 다시 썼다. 경영권 분쟁에 다시 불이 붙어 전날 급등했던 고려아연은 이날 현재 9%대 급락세다.코스닥지수는 장중 하락으로 전환했다. 이 시각 현재 코스닥지수는 0.42% 떨어진 726.44를 기록 중이다.'}

In [25]:
news_dict2 = {
    'title' : news_title,
    'content' : news_content
}
news_dict2

{'title': "'외인 팔자' 코스피, 상승폭 반납…2570선 보합권 등락",
 'content': '사진=연합뉴스13일 코스피지수가 외국인 투자자의 팔자에 장중 상승폭을 반납했다.이날 오후 1시18분 현재 코스피지수는 전일 대비 0.92포인트(0.04%) 오른 2575.74를 기록 중이다. 0.9% 상승 출발한 코스피지수는 이날 상승폭을 2600선까지 높이기도 했으나 외인과 금융투자에서 매물이 출회되면서 상단이 제한됐다.이 시각 현재 외국인은 유가증권시장에서 995억원 순매도 중이다. 금융투자도 1312억원 팔고 있다. 개인은 149억원 매수우위다. 기관은 3억원 매수우위다.코스피 시가총액 상위기업들은 혼조세다. SK하이닉스가 장중 하락으로 전환한 가운데 LG에너지솔루션, 삼성바이오로직스, 현대차, POSCO홀딩스가 내림세다. 반면 삼성전자, 기아, 네이버 등은 오르고 있다.한화에어로스페이스는 증권가 목표가 상향에 6%대 강세다. LS는 호반그룹이 지분을 매입했다는 소식에 18%대 뛰고 있다. 현대로템은 유럽 방위비 증강 기대감에 신고가를 다시 썼다. 경영권 분쟁에 다시 불이 붙어 전날 급등했던 고려아연은 이날 현재 9%대 급락세다.코스닥지수는 장중 하락으로 전환했다. 이 시각 현재 코스닥지수는 0.42% 떨어진 726.44를 기록 중이다.'}

In [26]:
import time

In [27]:
# 데이터를 저장할 수 있는 빈 리스트 생성
result = []
base_url = "https://finance.naver.com"
# href_list를 이용하여 반복문을 실행
for href in href_list:
    # base_url + href_list[index]를 이용하여 webdriver에 요청
    driver.get(base_url + href)
    # 2초 정도의 딜레이
    time.sleep(2)
    # 해당 driver에서 html을 추출하여 데이터 파싱
    news_soup = bs(driver.page_source, 'html.parser')
    # h2태그 중 id가 title_area인 태그에서 텍스트를 추출하여 변수에 저장
    news_title = news_soup.find(
        'h2',
        attrs = {'id' : 'title_area'}
    ).get_text()
    # div태그 중 id가 newsct_article인 태그에서 텍스트를 추출하여 변수에 저장
    news_content = news_soup.find(
        'div',
        attrs = {'id' : 'newsct_article'}
    ).get_text().replace('\n','').replace('\xa0','').strip()
    # news_dict 생성하여 h2태그의 텍스트와 div태그의 텍스트를 저장
    news_dict = {
        'title' : news_title, 'content' : news_content
    }
    # reseult에 news_dict를 추가
    result.append(news_dict)


In [28]:
result

[{'title': "'외인 팔자' 코스피, 상승폭 반납…2570선 보합권 등락",
  'content': '사진=연합뉴스13일 코스피지수가 외국인 투자자의 팔자에 장중 상승폭을 반납했다.이날 오후 1시18분 현재 코스피지수는 전일 대비 0.92포인트(0.04%) 오른 2575.74를 기록 중이다. 0.9% 상승 출발한 코스피지수는 이날 상승폭을 2600선까지 높이기도 했으나 외인과 금융투자에서 매물이 출회되면서 상단이 제한됐다.이 시각 현재 외국인은 유가증권시장에서 995억원 순매도 중이다. 금융투자도 1312억원 팔고 있다. 개인은 149억원 매수우위다. 기관은 3억원 매수우위다.코스피 시가총액 상위기업들은 혼조세다. SK하이닉스가 장중 하락으로 전환한 가운데 LG에너지솔루션, 삼성바이오로직스, 현대차, POSCO홀딩스가 내림세다. 반면 삼성전자, 기아, 네이버 등은 오르고 있다.한화에어로스페이스는 증권가 목표가 상향에 6%대 강세다. LS는 호반그룹이 지분을 매입했다는 소식에 18%대 뛰고 있다. 현대로템은 유럽 방위비 증강 기대감에 신고가를 다시 썼다. 경영권 분쟁에 다시 불이 붙어 전날 급등했던 고려아연은 이날 현재 9%대 급락세다.코스닥지수는 장중 하락으로 전환했다. 이 시각 현재 코스닥지수는 0.42% 떨어진 726.44를 기록 중이다.'},
 {'title': "코스피, 변동성 확대…상승분 모조리 '반납'",
  'content': "선물·옵션만기일, 금융투자 매물 '속출'코스피가 상승분을 모두 반납하며 약세 전환했다.선물·옵션 동시 만기일을 맞아 변동성이 확대되는 모습인데, 금융투자를 중심으로 대거 매도 물량이 나오고 있다.13일 오후 12시52분 현재 코스피지수는 전거래일대비 0.31포인트(0.01%) 하락한 2,574.51을 기록중이다.개인과 기관이 각각 30억원, 150억원 소폭 매수 우위를 보이고 있는 가운데 외국인은 970억원 매도 우위다.미국의 물가 둔화에도 불구하고 관세를 둘러싼 불확실성이, 그간 상승폭이 컸던 업종과 종목을 중심으